In [1]:
from datasets import load_dataset
dataset = load_dataset("squad")
dataset

Found cached dataset squad (/Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [2]:
def add_end_of_text(example):
    example['question'] =  example['question'] + '<|endoftext|>'
    return example

dataset = dataset.remove_columns(['id', 'title', 'context', 'answers'])
dataset = dataset.map(add_end_of_text)

Loading cached processed dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-c8e447486d74d178.arrow
Loading cached processed dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-4cba412ea21154c2.arrow


In [3]:
dataset['train']['question'][:10]

['To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?<|endoftext|>',
 'What is in front of the Notre Dame Main Building?<|endoftext|>',
 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?<|endoftext|>',
 'What is the Grotto at Notre Dame?<|endoftext|>',
 'What sits on top of the Main Building at Notre Dame?<|endoftext|>',
 'When did the Scholastic Magazine of Notre dame begin publishing?<|endoftext|>',
 "How often is Notre Dame's the Juggler published?<|endoftext|>",
 'What is the daily student paper at Notre Dame called?<|endoftext|>',
 'How many student news papers are found at Notre Dame?<|endoftext|>',
 'In what year did the student paper Common Sense begin publication at Notre Dame?<|endoftext|>']

In [4]:
from transformers import AutoTokenizer
model_checkpoint = "distilgpt2" 

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [5]:
sequence = ("This tokenizer is being applied in CS197 at"
            "Harvard.<|endoftext|>")
tokens = tokenizer.tokenize(sequence)
print(tokens)

['This', 'Ġtoken', 'izer', 'Ġis', 'Ġbeing', 'Ġapplied', 'Ġin', 'ĠCS', '197', 'Ġat', 'Har', 'vard', '.', '<|endoftext|>']


In [6]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[1212, 11241, 7509, 318, 852, 5625, 287, 9429, 24991, 379, 13587, 10187, 13, 50256]


In [7]:
sequence = ("This tokenizer is being applied in CS197 at"
            "Harvard.<|endoftext|>")
tokenizer(sequence)

{'input_ids': [1212, 11241, 7509, 318, 852, 5625, 287, 9429, 24991, 379, 13587, 10187, 13, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["question"], truncation=True)

tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    num_proc=4,
    remove_columns=["question"]
)

Loading cached processed dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-8257facdc542f6f9.arrow


Loading cached processed dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-942d7cd6df7369d0.arrow


Loading cached processed dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-84102809ff775f81.arrow


Loading cached processed dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-3dc31669e653e480.arrow


Loading cached processed dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-75347940eb01c6ec.arrow


Loading cached processed dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-16d169bc43bed6fb.arrow


Loading cached processed dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-cab8af4e4aab6812.arrow


Loading cached processed dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-f5d15a61cbaa172f.arrow


In [9]:
block_size = 128

def group_texts(examples):
    # repeat concatenation for input_ids and other keys
    concatenated_examples = {k: sum(examples[k], []) for k in
                            examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size

    # populate each of input_ids and other keys 
    result = {
        k: [t[i : i + block_size] for i in range(0,
            total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    # add labels because we'll need it as the output
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Loading cached processed dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-b8ea253201a818c9.arrow


Loading cached processed dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-70a631e300c5585b.arrow


Loading cached processed dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-08e467c3e792f6a8.arrow


Loading cached processed dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-adf33513613f763c.arrow


Loading cached processed dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-8749fd2ff7383034.arrow


Loading cached processed dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-829779fc90b69879.arrow


Loading cached processed dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-0b1d74bfa5d15ecb.arrow


Loading cached processed dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-e1600d11f068ef27.arrow


In [10]:
print(lm_datasets['train']['input_ids'][0])

[2514, 4150, 750, 262, 5283, 5335, 7910, 1656, 287, 1248, 3365, 287, 406, 454, 8906, 4881, 30, 50256, 2061, 318, 287, 2166, 286, 262, 23382, 20377, 8774, 11819, 30, 50256, 464, 32520, 3970, 286, 262, 17380, 2612, 379, 23382, 20377, 318, 13970, 284, 543, 4645, 30, 50256, 2061, 318, 262, 10299, 33955, 379, 23382, 20377, 30, 50256, 2061, 10718, 319, 1353, 286, 262, 8774, 11819, 379, 23382, 20377, 30, 50256, 2215, 750, 262, 3059, 349, 3477, 11175, 286, 23382, 288, 480, 2221, 12407, 30, 50256, 2437, 1690, 318, 23382, 20377, 338, 262, 39296, 1754, 3199, 30, 50256, 2061, 318, 262, 4445, 3710, 3348, 379, 23382, 20377, 1444, 30, 50256, 2437, 867, 3710, 1705, 9473, 389, 1043, 379, 23382, 20377, 30, 50256, 818, 644, 614, 750, 262, 3710, 3348]


In [11]:
tokenizer.decode(lm_datasets['train']['input_ids'][0])

"To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?<|endoftext|>What is in front of the Notre Dame Main Building?<|endoftext|>The Basilica of the Sacred heart at Notre Dame is beside to which structure?<|endoftext|>What is the Grotto at Notre Dame?<|endoftext|>What sits on top of the Main Building at Notre Dame?<|endoftext|>When did the Scholastic Magazine of Notre dame begin publishing?<|endoftext|>How often is Notre Dame's the Juggler published?<|endoftext|>What is the daily student paper at Notre Dame called?<|endoftext|>How many student news papers are found at Notre Dame?<|endoftext|>In what year did the student paper"

In [12]:
small_train_dataset = \
    lm_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = \
    lm_datasets["validation"].shuffle(seed=42).select(range(100))

Loading cached shuffled indices for dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-da0977e44b1e20aa.arrow
Loading cached shuffled indices for dataset at /Users/alyssahuang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-54a888906a9b2ded.arrow


In [13]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

In [14]:
training_args = TrainingArguments(
    output_dir="./models"
    f"{model_checkpoint}-squad",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
)

trainer.train()

/Users/alyssahuang/Documents/GitHub/CS197/CS197 Pset2/./modelsdistilgpt2-squad is already a clone of https://huggingface.co/alyssahuang02/modelsdistilgpt2-squad. Make sure you pull the latest changes with `repo.git_pull()`.
/Users/alyssahuang/miniconda/envs/lec4/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 100
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 39
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME 

  0%|          | 0/39 [00:00<?, ?it/s]

***** Running Evaluation *****
  Num examples = 100
  Batch size = 8


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 3.718662977218628, 'eval_runtime': 16.9562, 'eval_samples_per_second': 5.898, 'eval_steps_per_second': 0.767, 'epoch': 1.0}


...

***** Running Evaluation *****
  Num examples = 100
  Batch size = 8


  0%|          | 0/13 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 3.486569881439209, 'eval_runtime': 14.0535, 'eval_samples_per_second': 7.116, 'eval_steps_per_second': 0.925, 'epoch': 3.0}
{'train_runtime': 209.2477, 'train_samples_per_second': 1.434, 'train_steps_per_second': 0.186, 'train_loss': 3.9001183143028846, 'epoch': 3.0}


TrainOutput(global_step=39, training_loss=3.9001183143028846, metrics={'train_runtime': 209.2477, 'train_samples_per_second': 1.434, 'train_steps_per_second': 0.186, 'train_loss': 3.9001183143028846, 'epoch': 3.0})

In [15]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 100
  Batch size = 8


  0%|          | 0/13 [00:00<?, ?it/s]

Perplexity: 32.67


In [16]:
tokenizer.save_pretrained(f"{model_checkpoint}-squad")
model.push_to_hub(f"{model_checkpoint}-squad")

tokenizer config file saved in distilgpt2-squad/tokenizer_config.json
Special tokens file saved in distilgpt2-squad/special_tokens_map.json
Configuration saved in distilgpt2-squad/config.json
Model weights saved in distilgpt2-squad/pytorch_model.bin
Uploading the following files to alyssahuang02/distilgpt2-squad: config.json,pytorch_model.bin


In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(f"rajpurkar/{model_checkpoint}-squad")
tokenizer = AutoTokenizer.from_pretrained(f"rajpurkar/{model_checkpoint}-squad")


Downloading:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

loading configuration file config.json from cache at /Users/alyssahuang/.cache/huggingface/hub/models--rajpurkar--distilgpt2-squad/snapshots/916b25d720098772e68eb60a73aa1e89886d1c0c/config.json
Model config GPT2Config {
  "_name_or_path": "rajpurkar/distilgpt2-squad",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "su

Downloading:   0%|          | 0.00/334M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /Users/alyssahuang/.cache/huggingface/hub/models--rajpurkar--distilgpt2-squad/snapshots/916b25d720098772e68eb60a73aa1e89886d1c0c/pytorch_model.bin
All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at rajpurkar/distilgpt2-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.


Downloading:   0%|          | 0.00/261 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

loading file vocab.json from cache at /Users/alyssahuang/.cache/huggingface/hub/models--rajpurkar--distilgpt2-squad/snapshots/916b25d720098772e68eb60a73aa1e89886d1c0c/vocab.json
loading file merges.txt from cache at /Users/alyssahuang/.cache/huggingface/hub/models--rajpurkar--distilgpt2-squad/snapshots/916b25d720098772e68eb60a73aa1e89886d1c0c/merges.txt
loading file tokenizer.json from cache at /Users/alyssahuang/.cache/huggingface/hub/models--rajpurkar--distilgpt2-squad/snapshots/916b25d720098772e68eb60a73aa1e89886d1c0c/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /Users/alyssahuang/.cache/huggingface/hub/models--rajpurkar--distilgpt2-squad/snapshots/916b25d720098772e68eb60a73aa1e89886d1c0c/special_tokens_map.json
loading file tokenizer_config.json from cache at /Users/alyssahuang/.cache/huggingface/hub/models--rajpurkar--distilgpt2-squad/snapshots/916b25d720098772e68eb60a73aa1e89886d1c0c/tokenizer_config.json


In [18]:
start_text = ("A speedrun is a playthrough of a video game, \
or section of a video game, with the goal of \
completing it as fast as possible. Speedruns \
often follow planned routes, which may incorporate sequence \
breaking, and might exploit glitches that allow sections to \
be skipped or completed more quickly than intended. ")

prompt = "What is the"
inputs = tokenizer(
     start_text + prompt,
     add_special_tokens=False,
     return_tensors="pt")["input_ids"]

prompt_length = len(tokenizer.decode(inputs[0]))
outputs = model.generate(
     inputs,
     max_length=100,
     do_sample=True,
     top_k=50,
     top_p=0.95,
     temperature=0.9,
     num_return_sequences=3)

generated = prompt + tokenizer.decode(outputs[0])[prompt_length + 1:]

print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A speedrun is a playthrough of a video game, or section of a video game, with the goal of completing it as fast as possible. Speedruns often follow planned routes, which may incorporate sequence breaking, and might exploit glitches that allow sections to be skipped or completed more quickly than intended. What is the goal of Speedrun?<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>
